<a href="https://colab.research.google.com/github/SayantikaFSU/ocp-ci-analysis/blob/master/TitanicDataset_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'titanic:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3136%2F26502%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240518%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240518T154025Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D78fa71ac86018f8e9f546b1008412df4de77d9d58d4784468ceee50c871c2a30e7f3c58460c8017d062929c55c040e0cd429b3700eadb7103d505f22b3ea1f391e66297ec9323277c1989bbf7bed51110781c7eacbc898cb47cc7918c1988faf4e132df56d6996d6248494fbb4632e72f1efb5047969bd2434f942c3a358b70d242c158a9fce4c66797f709c7b7c85a9d14035a8c3ea11557284ae70bea45f37bf1a3ba57b07cb1c18cc6996fcd6b59a938c46a91de2a14bef168cc81fef8a7f0238e9be89a8c5707109521237693fb07414f1deb402758ad820f3d495a20ffea82c818fc122c2effd6710b7bbdee9311d412eec08708d7f6bb1c54450d088ff'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV
import re

In [ ]:
import pandas as pd

test_path='/kaggle/input/titanic/test.csv'
train_path='/kaggle/input/titanic/train.csv'


#print(train.head(2),test.head(2))
#print(train.shape)
#print(train.columns)
#print(train.head(5))
#print(train.info())


Exploratory Data Analysis:
*******************************************************

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv(train_path)
df

In [ ]:
#### (1) Pie chart showing the percentage of survived vs dead:


# Count the occurrences of each category
category_counts = df['Survived'].value_counts()

# Plot a pie chart
plt.pie(category_counts, labels=category_counts.index, autopct='%1.1f%%', startangle=90, colors=['skyblue', 'lightcoral'])
plt.title('PiePlot of Survived(1) vs Not Survived(0)')
plt.show()

In [ ]:
###(2) 'Sex' wise survival rate and bar plot:

# Total females
totalFemales = len(df[df.Sex == 'female'])
# Total males
totalMales = len(df[df.Sex == 'male'])

# We added total 1s in the 'Survived' col each of female and male category
femalesSurvived = df.loc[df.Sex == 'female']['Survived'].values.sum()
malesSurvived = df.loc[df.Sex == 'male']['Survived'].values.sum()

print('Total females survived: ', femalesSurvived, '/', totalFemales, 'i.e', round(femalesSurvived*100/totalFemales,2), '% survival rate')
print('Total males survived: ', malesSurvived, '/', totalMales, 'i.e', round(malesSurvived*100/totalMales,2), '% survival rate')



sns.countplot(df, x= "Sex", hue='Survived')
plt.title("'Sex'-wise survival rate")

In [ ]:
#### (3) Embarked wise Survived plot

sns.countplot(df, x= "Embarked", hue='Survived')
plt.title("'Embarked'-wise survival rate")

In [ ]:
#### (4) Class wise survival Rate

sns.countplot(df, x= "Pclass", hue='Survived')
plt.title("PClass-wise survival rate")

In [ ]:
### (5) Fare on Survival Rate:
sns.set(style="whitegrid")
fig = plt.figure(figsize=(8,5))
ax=sns.histplot(df.loc[(df['Survived'] == 0),'Fare'] ,bins=10, color= '#557C55',label='not survived',kde=True)
ax=sns.histplot(df.loc[(df['Survived'] == 1),'Fare'] ,bins=10, color= '#7B68EE', label='survived', kde=True)

plt.title('Fare Distribution vs Survival Rate ')
plt.xlabel("Fare")
plt.ylabel('Frequency');
plt.legend()
plt.show()

In [ ]:
## (6) Age on Survival Rate:
sns.set(style="whitegrid")
fig = plt.figure(figsize=(8,5))
ax=sns.histplot(df.loc[(df['Survived'] == 0),'Age'] ,bins=10, color= '#557C55',label='not survived',kde=True)
ax=sns.histplot(df.loc[(df['Survived'] == 1),'Age'] ,bins=10, color= '#7B68EE', label='survived', kde=True)

plt.title('Age Distribution vs Survival Rate ')
plt.xlabel("Age")
plt.ylabel('Frequency');
plt.legend()
plt.show()

In [ ]:
###(6)Pair Plot:

pair_plt=sns.pairplot(df,hue='Survived',palette = 'seismic')

pair_plt.fig.suptitle('Pair Plot of Survived vs Other Numerical Predictors', y=1.02, fontsize=14)

Data Preprocessing:
****************************************************************

In [ ]:
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV
import re
from xgboost import XGBClassifier
import warnings

In [ ]:
# Ignore all warnings
warnings.filterwarnings("ignore")

# ==================== Functions ============================ #
# This section contains various functions used in the project #
# These all perform specific tasks                            #
# =========================================================== #


def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""


In [ ]:
def data_preprocessing_train(train_path):

    data=pd.read_csv(train_path)

    #(1):convert Sex to numeric:
    data['Sex'] = data['Sex'].map({'male': 1, 'female': 0})

    #(2)set embarked to S if not stated:
    data['Embarked'] = data['Embarked'].fillna('S')
    data['Embarked'] = data['Embarked'].map({'Q': 2, 'S': 1, 'C': 0})

     #(3)feature engineering: add feature that tells whether a passenger had a cabin on the Titanic
    #use this since basin numbers are not usable
    data['Has_Cabin'] = data["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

    #(4) Create new feature FamilySize as a combination of SibSp and Parch
    data['Family_Size'] = data['SibSp'] + data['Parch'] + 1

    #(5) Create new feature IsAlone from FamilySize
    data['IsAlone'] = 0
    data.loc[data['Family_Size'] == 1, 'IsAlone'] = 1

    #(6)Fare Bins pushed into 4 quantiles
    ''' CategoricalFare  Survived
    0       [0, 7.91]  0.197309
    1  (7.91, 14.454]  0.303571
    2    (14.454, 31]  0.454955
    3   (31, 512.329]  0.581081
    '''
    data.loc[ data['Fare'] <= 7.91, 'Fare'] = 0
    data.loc[(data['Fare'] > 7.91) & (data['Fare'] <= 14.454), 'Fare'] = 1
    data.loc[(data['Fare'] > 14.454) & (data['Fare'] <= 31), 'Fare']   = 2
    data.loc[ data['Fare'] > 31, 'Fare']  = 3
    data['Fare'] = data['Fare'].astype(int)

     #(7) Create a new feature Title, containing the titles of passenger names. Replace rare titles with "rare"
    #replace and fix miss spellings
    #map str to intergrers and fill none with 0
    '''
    Capt           0     1
    Col            0     2
    Countess       1     0
    Don            0     1
    Dr             1     6
    Jonkheer       0     1
    Lady           1     0
    Major          0     2
    Master         0    40
    Miss         182     0
    Mlle           2     0
    Mme            1     0
    Mr             0   517
    Mrs          125     0
    Ms             1     0
    Rev            0     6
    Sir            0     1
    '''
    data['Title'] = data['Name'].apply(get_title)
    data['Title'] = data['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    data['Title'] = data['Title'].replace('Mlle', 'Miss')
    data['Title'] = data['Title'].replace('Ms', 'Miss')
    data['Title'] = data['Title'].replace('Mme', 'Mrs')
    data['Title'] = data['Title'].map({"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5})
    data['Title'] = data['Title'].fillna(0)



    #(8)drop all cases without age`
    data = data.dropna(subset=['Age'])

    #drop non usable variables
    data = data.drop('Ticket', axis=1)
    data = data.drop('Cabin', axis=1)
    data = data.drop('Name', axis=1)
    data = data.drop('PassengerId', axis=1)



    #(9)drop any rows with missing data
    data = data.dropna()

    print(data.info())


    return(data)



In [ ]:
def data_preprocessing_test(test_path):

    data=pd.read_csv(test_path)

    #(1)convert Sex to numeric
    data['Sex'] = data['Sex'].map({'male': 1, 'female': 0})

    #(2)set embarked to S if not stated
    data['Embarked'] = data['Embarked'].fillna('S')
    data['Embarked'] = data['Embarked'].map({'Q': 2, 'S': 1, 'C': 0})

    #(3)feature engineering: add feature that tells whether a passenger had a cabin on the Titanic
    #use this since basin numbers are not usable
    data['Has_Cabin'] = data["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

    # (4)Create new feature FamilySize as a combination of SibSp and Parch
    data['Family_Size'] = data['SibSp'] + data['Parch'] + 1

    #(5) Create new feature IsAlone from FamilySize
    data['IsAlone'] = 0
    data.loc[data['Family_Size'] == 1, 'IsAlone'] = 1

    #(6)Fare Bins pushed into 4 quantiles
    ''' CategoricalFare  Survived
    0       [0, 7.91]  0.197309
    1  (7.91, 14.454]  0.303571
    2    (14.454, 31]  0.454955
    3   (31, 512.329]  0.581081
    '''
    data.loc[ data['Fare'] <= 7.91, 'Fare'] = 0
    data.loc[(data['Fare'] > 7.91) & (data['Fare'] <= 14.454), 'Fare'] = 1
    data.loc[(data['Fare'] > 14.454) & (data['Fare'] <= 31), 'Fare']   = 2
    data.loc[ data['Fare'] > 31, 'Fare']  = 3
    #drop all cases without fare`
    data = data.dropna(subset=['Fare'])
    data['Fare'] = data['Fare'].astype(int)

    # (7)Create a new feature Title, containing the titles of passenger names. Replace rare titles with "rare"
    #replace and fix miss spellings
    #map str to intergrers and fill none with 0
    '''
    Capt           0     1
    Col            0     2
    Countess       1     0
    Don            0     1
    Dr             1     6
    Jonkheer       0     1
    Lady           1     0
    Major          0     2
    Master         0    40
    Miss         182     0
    Mlle           2     0
    Mme            1     0
    Mr             0   517
    Mrs          125     0
    Ms             1     0
    Rev            0     6
    Sir            0     1
    '''
    data['Title'] = data['Name'].apply(get_title)
    data['Title'] = data['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    data['Title'] = data['Title'].replace('Mlle', 'Miss')
    data['Title'] = data['Title'].replace('Ms', 'Miss')
    data['Title'] = data['Title'].replace('Mme', 'Mrs')
    data['Title'] = data['Title'].map({"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5})
    data['Title'] = data['Title'].fillna(0)



    #(8)drop all cases without age`
    data = data.dropna(subset=['Age'])
    #drop non usable variables
    data = data.drop('Ticket', axis=1)
    data = data.drop('Cabin', axis=1)
    data = data.drop('Name', axis=1)



    #(10)drop any rows with missing data
    data = data.dropna()

    # (11) Add empty column for 'Predicted Survived'

    data['Survived']=pd.Series(dtype='float64')

    return(data)
    #return(test_data, X_test, Y_test)



In [ ]:
train_data=data_preprocessing_train(train_path)


In [ ]:
predictor_columns=[ 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Has_Cabin',
       'IsAlone', 'Title']
response_column=['Survived']

X=train_data[predictor_columns]
Y=train_data[response_column]

X.shape, Y.shape

In [ ]:
# Check Correlation:


import seaborn as sns
import matplotlib.pyplot as plt

traindata=train_data[['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Has_Cabin',
       'IsAlone', 'Title','Survived']]

# Calculate the covariance + Correlation matrix:
cov_matrix = np.cov(traindata, rowvar=False)
correlation_matrix = traindata.corr()

# Create a heatmap of the covariance matrix
plt.figure(figsize=(10, 6))
#sns.heatmap(cov_matrix, annot=True, fmt="0.2f",cmap='coolwarm', linewidths=0.5)

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)

# Set the title
plt.title('Correlation Matrix Heatmap',fontsize= 14)

# Customize tick labels
plt.xticks(np.arange(len(traindata.columns)), traindata.columns, rotation=45,fontsize=8)
plt.yticks(np.arange(len(traindata.columns)), traindata.columns, rotation=0,fontsize=8)

# Show the plot
plt.show()

Models to compare:
*****************************************************

In [ ]:
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_selection import RFECV


In [ ]:
# Model 1: Logistic Regression without feature selection

from sklearn.linear_model import LogisticRegression


# Split the data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# Create and train the Logistic Regression model
logreg_model = LogisticRegression()
logreg_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = logreg_model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)
classification_rep = classification_report(y_val, y_pred)
mse_logit=mean_squared_error(y_val,y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)
print(f"MSE for logistic classifier {mse_logit:0.3f}")


In [ ]:
# Model 2: Logistic regression with feature selection

# Split the data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)



# Create the Logistic Regression model
logreg_model = LogisticRegression()


# Create the RFECV model with cross-validation
rfecv = RFECV(estimator=logreg_model, step=1, cv=StratifiedKFold(5), scoring='accuracy')


# Fit the RFECV model to the training data
rfecv.fit(X_train, Y_train)

# Get the optimal number of features
optimal_num_features = rfecv.n_features_

# Print the selected features
selected_features = pd.DataFrame({'Feature': X.columns, 'Selected': rfecv.support_, 'Ranking': rfecv.ranking_})
print(f"Selected Features:")
print(selected_features)


# Transform the training and testing sets to keep only the selected features
X_train_selected = rfecv.transform(X_train)
X_val_selected = rfecv.transform(X_val)

# Train the XGBoost model on the selected features
logreg_model.fit(X_train_selected, Y_train)

# Make predictions on the test set
y_pred = logreg_model.predict(X_val_selected)

# Evaluate the model
accuracy = accuracy_score(Y_val, y_pred)
conf_matrix = confusion_matrix(Y_val, y_pred)
classification_rep = classification_report(Y_val, y_pred)
mse_logreg=mean_squared_error(Y_val,y_pred)

print("Optimal Number of Features:", optimal_num_features)
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)
print(f"MSE for LogREg Classifier = {mse_logreg:0.3f}")


log_reg_selected_feature=selected_features.loc[selected_features['Ranking']==1,'Feature']
log_reg_selected_feature

In [ ]:
# Model 2: XGBOOST with feature elimination:
from xgboost import XGBClassifier

# Split the data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)



# Create the Logistic Regression model
xgb_model = XGBClassifier()


# Create the RFECV model with cross-validation
rfecv = RFECV(estimator=xgb_model, step=1, cv=StratifiedKFold(5), scoring='accuracy')


# Fit the RFECV model to the training data
rfecv.fit(X_train, Y_train)

# Get the optimal number of features
optimal_num_features = rfecv.n_features_

# Print the selected features
selected_features = pd.DataFrame({'Feature': X.columns, 'Selected': rfecv.support_, 'Ranking': rfecv.ranking_})
print(f"Selected Features:")
print(selected_features)


# Transform the training and testing sets to keep only the selected features
X_train_selected = rfecv.transform(X_train)
X_val_selected = rfecv.transform(X_val)

# Train the XGBoost model on the selected features
xgb_model.fit(X_train_selected, Y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_val_selected)

# Evaluate the model
accuracy = accuracy_score(Y_val, y_pred)
conf_matrix = confusion_matrix(Y_val, y_pred)
classification_rep = classification_report(Y_val, y_pred)
mse_xgb=mean_squared_error(Y_val,y_pred)

print("Optimal Number of Features:", optimal_num_features)
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)
print(f"MSE for XGB Classifier = {mse_xgb:0.3f}")

xgb_selected_feature=selected_features.loc[selected_features['Ranking']==1,'Feature']
print(pd.DataFrame({'Selected Feature': xgb_selected_feature,'Importance':xgb_model.feature_importances_}))

In [ ]:
# Model 3: Random Forrest with feature elimination:
from sklearn.ensemble import RandomForestClassifier

# Split the data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)



# Create the Logistic Regression model
rfc_model = RandomForestClassifier()


# Create the RFECV model with cross-validation
rfecv = RFECV(estimator=rfc_model, step=1, cv=StratifiedKFold(5), scoring='accuracy')


# Fit the RFECV model to the training data
rfecv.fit(X_train, Y_train)

# Get the optimal number of features
optimal_num_features = rfecv.n_features_

# Print the selected features
selected_features = pd.DataFrame({'Feature': X.columns, 'Selected': rfecv.support_, 'Ranking': rfecv.ranking_})
print(f"Selected Features:")
print(selected_features)


# Transform the training and testing sets to keep only the selected features
X_train_selected = rfecv.transform(X_train)
X_val_selected = rfecv.transform(X_val)

# Train the XGBoost model on the selected features
rfc_model.fit(X_train_selected, Y_train)

# Make predictions on the test set
y_pred = rfc_model.predict(X_val_selected)

# Evaluate the model
accuracy = accuracy_score(Y_val, y_pred)
conf_matrix = confusion_matrix(Y_val, y_pred)
classification_rep = classification_report(Y_val, y_pred)
mse_rfc=mean_squared_error(Y_val,y_pred)

print("Optimal Number of Features:", optimal_num_features)
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)
print(f"MSE for RandomForrest Classifier = {mse_rfc:0.3f}")

rfe_selected_feature=selected_features.loc[selected_features['Ranking']==1,'Feature']
#print(rfe_selected_feature)

print(pd.DataFrame({'Selected Feature': rfe_selected_feature,'Importance':rfc_model.feature_importances_}))

Hyper parameter Tuning:
*****************************************

In [ ]:
# Model Logistic Regression

from sklearn.model_selection import train_test_split, GridSearchCV

X_selected= train_data[log_reg_selected_feature]

# Split the data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X_selected, Y, test_size=0.2, random_state=42)

# Create the Logistic Regression model
logreg_model = LogisticRegression()

# Define the parameter grid for grid search
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [100, 200, 300]
}

# Create the grid search object
grid_search = GridSearchCV(estimator=logreg_model, param_grid=param_grid, cv=StratifiedKFold(5), scoring='accuracy')

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the Logistic Regression model with the best hyperparameters
best_logreg_model = LogisticRegression(**best_params)
best_logreg_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_logreg_model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)
classification_rep = classification_report(y_val, y_pred)
mse_logreg=mean_squared_error(y_val,y_pred)

print("Best Hyperparameters:", best_params)
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)
print(f"MSE for best logreg Classifier={mse_logreg:0.3f}")



In [ ]:
# Model Random Forrest Classifier:

X_selected= train_data[rfe_selected_feature]


# Split the data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X_selected, Y, test_size=0.2, random_state=42)

# Create the Random Forest Classifier model
rf_model = RandomForestClassifier()

# Define the parameter grid for grid search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create the grid search object
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=StratifiedKFold(5), scoring='accuracy')

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the Random Forest model with the best hyperparameters on the selected features
best_rf_model = RandomForestClassifier(**best_params)
best_rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_rf_model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)
classification_rep = classification_report(y_val, y_pred)
mse_rf=mean_squared_error(y_val,y_pred)

print("Best Hyperparameters:", best_params)
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)
print(f"MSE for best RF model={mse_rf:0.3f}")

In [ ]:
# Model XGB Classifier:

X_selected= train_data[xgb_selected_feature]


# Split the data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X_selected, Y, test_size=0.2, random_state=42)


# Create an XGBoost classifier
xgb_model = XGBClassifier()

# Define a parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
}

# Create the grid search object
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=StratifiedKFold(5), scoring='accuracy')

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the Random Forest model with the best hyperparameters on the selected features
best_xgb_model = XGBClassifier(**best_params)
best_xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_xgb_model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)
classification_rep = classification_report(y_val, y_pred)
mse_xgb=mean_squared_error(y_val,y_pred)

print("Best Hyperparameters:", best_params)
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)
print(f"MSE for best XGB model={mse_xgb:0.3f}")

Fitting the Model on the test Set:
*****************************************************************

In [ ]:
test_data=data_preprocessing_test(test_path)
test_data.shape
test_data.columns

(1)Fitting XGBOOST:
****************

In [ ]:


#prepare the test data predictors based on the features selected by the best chosen model
X_test = test_data[ xgb_selected_feature]
X_test.head()

In [ ]:
Y_test=best_xgb_model.predict(X_test)


In [ ]:
# Prep the result table with passenger Id and Survival:

test_data['Survived']=Y_test
result_table=test_data[['PassengerId','Survived']]
result_table

In [ ]:
import matplotlib.pyplot as plt


# Create a bar plot
plt.figure(figsize=(8, 6))

# Count the occurrences of each category
category_counts = result_table['Survived'].value_counts()


ax=category_counts.plot(kind='barh', color='skyblue')

# Change category names on the x-axis
category_names = ['Not Survived', 'survived']
ax.set_yticklabels(category_names, rotation=0)

# Annotate each bar with its height
#for p in ax.patches:
    #ax.annotate(str(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()),
                #ha='center', va='center', xytext=(0, 10), textcoords='offset points')

# Annotate each bar with its width
for p in ax.patches:
    ax.annotate(str(p.get_width()), (p.get_width(), p.get_y() + p.get_height() / 2.),
                ha='center', va='center', xytext=(10, 0), textcoords='offset points')


# Add labels and title
plt.ylabel('Survived')
plt.xlabel('Count')
plt.title('Bar Plot of Survived Counts: XGBOOST Model')

# Show the plot
plt.show()

(2) Fitting Random Forrest Classifier model:
*********************************************************************

In [ ]:
#prepare the test data predictors based on the features selected by the best chosen model
X_test = test_data[ rfe_selected_feature]
X_test.head()

In [ ]:
Y_test=best_rf_model.predict(X_test)

# Prep the result table with passenger Id and Survival:

test_data['Survived']=Y_test
result_table=test_data[['PassengerId','Survived']]
result_table

In [ ]:
import matplotlib.pyplot as plt


# Count the occurrences of each category
category_counts = result_table['Survived'].value_counts()

# Create a bar plot
plt.figure(figsize=(8, 6))

ax=category_counts.plot(kind='barh', color='skyblue')

# Change category names on the x-axis
category_names = ['Not Survived', 'survived']
ax.set_yticklabels(category_names, rotation=0)

# Annotate each bar with its height
#for p in ax.patches:
    #ax.annotate(str(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()),
                #ha='center', va='center', xytext=(0, 10), textcoords='offset points')

# Annotate each bar with its width
for p in ax.patches:
    ax.annotate(str(p.get_width()), (p.get_width(), p.get_y() + p.get_height() / 2.),
                ha='center', va='center', xytext=(10, 0), textcoords='offset points')


# Add labels and title
plt.ylabel('Survived')
plt.xlabel('Count')
plt.title('Bar Plot of Survived Counts: Random Forrest Classifier Model')

# Show the plot
plt.show()

This section deals with the case if we have considered the whole train dataset, not divided into validation and training subsets.
******************************************************************************

In [ ]:
#Random Forrest Model:
X_rf=train_data[rfe_selected_feature]

best_rf_model.fit(X_rf,Y)

# Make predictions on the test set
y_pred = best_rf_model.predict(X_rf)

# Evaluate the model
accuracy = accuracy_score(Y, y_pred)
conf_matrix = confusion_matrix(Y, y_pred)
classification_rep = classification_report(Y, y_pred)
mse_xgb=mean_squared_error(Y,y_pred)

print('Best RF Model:\n', best_rf_model)
#print("Best Hyperparameters:\n", best_params)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)
print(f"MSE for best RF model={mse_xgb:0.3f}")

In [ ]:
# XGB Model
X_xgb=train_data[xgb_selected_feature]

best_xgb_model.fit(X_xgb,Y)

# Make predictions on the test set
y_pred = best_xgb_model.predict(X_xgb)

# Evaluate the model
accuracy = accuracy_score(Y, y_pred)
conf_matrix = confusion_matrix(Y, y_pred)
classification_rep = classification_report(Y, y_pred)
mse_xgb=mean_squared_error(Y,y_pred)

print('Best XGB Model:', best_xgb_model)
#print("Best Hyperparameters:", best_params)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)
print(f"MSE for best XGB model={mse_xgb:0.3f}")